In [1]:
import pandas as pd
import numpy as np
import requests
import json
from keys import gkey

In [2]:
prefix = "data/__harvest3r_twitter_data_0"
postfix = "-04_0.json"
data = pd.DataFrame()
for i in np.arange(5):
    file_name = prefix + str(i+1) + postfix
    print("Reading " + file_name + "\n");
    df = pd.read_json(file_name)
    print(df.shape)
    data = pd.concat([data, df])

Reading data/__harvest3r_twitter_data_01-04_0.json

(25333, 5)
Reading data/__harvest3r_twitter_data_02-04_0.json

(21270, 5)
Reading data/__harvest3r_twitter_data_03-04_0.json

(21466, 5)
Reading data/__harvest3r_twitter_data_04-04_0.json

(24843, 5)
Reading data/__harvest3r_twitter_data_05-04_0.json

(25994, 5)


In [3]:
print(data.shape)
data.head()

(118906, 5)


,lang,main,published,sentiment,source_location
0,de,Aktuellste Immoangebote http://tinyurl.com/cth...,2016-04-01T10:07:39Z,NEUTRAL,Bremgarten
1,de,Du bist 6 - 14 Jahre alt & sprichst Schweizerd...,2016-04-01T09:47:26Z,NEUTRAL,Bad Zurzach
2,fr,EN DIRECT sur #Periscope : Avec awa la blg htt...,2016-04-01T21:50:35Z,NEUTRAL,Ginevra
3,en,@ZHA_News https://youtu.be/UxQmxIm4q1s Rest i...,2016-04-01T02:35:56Z,POSITIVE,Lucerna
4,de,Unterwegs rastet der Velofahrer gerne bei eine...,2016-04-01T14:08:57Z,NEUTRAL,Hochdorf


In [4]:
locations = pd.unique(data.source_location)
print(locations.shape)
locations

(127,)


array(['Bremgarten', 'Bad Zurzach', 'Ginevra', 'Lucerna', 'Hochdorf',
       'Ennetbürgen', 'Suica', 'Arth', 'Conthey', 'Langnau', 'Einsiedeln',
       'Porrentruy', 'Poschiavo', 'Saanen', 'Visp', 'Zwitserland',
       'Rheinfelden', 'Glarus', 'Herisau', 'Laufen', 'Buchs', 'Losanna',
       'Altdorf', 'Romanshorn', 'Sursee', 'Schmerikon', 'Schwyz', 'Scuol',
       'Lauterbrunnen', 'Brugg', 'Weinfelden', 'Szwajcaria', 'Arbon',
       'Zofingen', 'Kreuzlingen', 'سويسرا', 'Stans', 'Schenkon', 'Bâle',
       'Affoltern am Albis', 'Zermatt', 'Genf', 'Confoederatio Helvetica',
       'Швейцария', 'Aigle', 'スイス連邦', 'Liestal', 'Suïssa', 'Sarnen',
       'Monthey', 'Bulle', 'Uster', 'Biel/Bienne', 'Zurigo', 'Olten',
       'Frauenfeld', 'Delémont', 'Bülach', 'Zuerich', 'Interlaken',
       'Waldenburg', 'Solothurn', 'İsviçre', 'Bellinzona', 'Neuchâtel',
       '스위스', 'Aarau', 'Sankt Gallen', 'Thun', 'Bienne', 'Suíça', 'Chur',
       'Lucerne', 'Svajcarska', 'Brig', 'Marly', 'Helvetia', 'スイス',
 

In [5]:
# mapping location -> canton using google api
# function taking the name of the place and returning the long address of it
def getAddress(name):
  glink="https://maps.googleapis.com/maps/api/place/textsearch/json?query={}&key={}".format(name + " Switzerland", gkey)
  r = requests.get(glink)
  dic = json.loads(r.text)
  try:
    return dic['results'][0]['formatted_address']
  except:
    return ''

In [6]:
getAddress('Zürich')

'Zürich, Switzerland'

In [7]:
geocodeLink = "https://maps.googleapis.com/maps/api/geocode/json?sensor=true"
from time import sleep

# Get the resulting Json file from the request, and put it in a json dictionary
def getGeocodeJsonDictionary(name):
    link =  u"{}&address=[{}]".format(geocodeLink, name, gkey)
    sleep(0.2) # a workaround for the rate-limit of Google geocode API
    r = requests.get(link)
    dic = json.loads(r.text)
    return dic

# Get the canton from the administrative_area_level_1 field in the dictionary
def getCantonFromGeocodeDictionary(dic):
    a = dic['results'][0]['address_components']
    canton=[x['short_name'] for x in a if x['types'][0]=='administrative_area_level_1'][0]   
    return canton

# Get the canton by composing the two previous functions
def getCanton(name):
    return getCantonFromGeocodeDictionary(getGeocodeJsonDictionary(name))

In [8]:
addresses = [[a, getAddress(a)] for a in locations]

In [9]:
addresses

[['Bremgarten', '5620 Bremgarten, Switzerland'],
 ['Bad Zurzach', '5330 Bad Zurzach, Switzerland'],
 ['Ginevra', 'Geneva, Switzerland'],
 ['Lucerna', 'Lucerne, Switzerland'],
 ['Hochdorf', 'Hochdorf, Switzerland'],
 ['Ennetbürgen', 'Ennetbürgen, Switzerland'],
 ['Suica', 'Switzerland'],
 ['Arth', 'Arth, Switzerland'],
 ['Conthey', 'Conthey, Switzerland'],
 ['Langnau', 'Langnau im Emmental, Switzerland'],
 ['Einsiedeln', 'Einsiedeln, Switzerland'],
 ['Porrentruy', 'Porrentruy, Switzerland'],
 ['Poschiavo', 'Poschiavo, Switzerland'],
 ['Saanen', 'Saanen, Switzerland'],
 ['Visp', '3930 Visp, Switzerland'],
 ['Zwitserland', 'Switzerland'],
 ['Rheinfelden', 'Rheinfelden, Switzerland'],
 ['Glarus', '8750 Glarus, Switzerland'],
 ['Herisau', 'Herisau, Switzerland'],
 ['Laufen', '4242 Laufen, Switzerland'],
 ['Buchs', 'Buchs, Switzerland'],
 ['Losanna', 'Lausanne, Switzerland'],
 ['Altdorf', '6460 Altdorf, Switzerland'],
 ['Romanshorn', '8590 Romanshorn, Switzerland'],
 ['Sursee', 'Sursee, Swit

In [14]:
def add(a):
    if a[1] == '':
        if a[0] == 'Suisse' :
            val = 'Switzerland'
        else :
            val = a[0]
    else :
        val = a[1]
    return val

addresses = [[a[0], add(a)] for a in addresses]
addresses = [a for a in addresses if a[1] != 'Switzerland']

In [15]:
addresses

[['Bremgarten', '5620 Bremgarten, Switzerland'],
 ['Bad Zurzach', '5330 Bad Zurzach, Switzerland'],
 ['Ginevra', 'Geneva, Switzerland'],
 ['Lucerna', 'Lucerne, Switzerland'],
 ['Hochdorf', 'Hochdorf, Switzerland'],
 ['Ennetbürgen', 'Ennetbürgen, Switzerland'],
 ['Arth', 'Arth, Switzerland'],
 ['Conthey', 'Conthey, Switzerland'],
 ['Langnau', 'Langnau im Emmental, Switzerland'],
 ['Einsiedeln', 'Einsiedeln, Switzerland'],
 ['Porrentruy', 'Porrentruy, Switzerland'],
 ['Poschiavo', 'Poschiavo, Switzerland'],
 ['Saanen', 'Saanen, Switzerland'],
 ['Visp', '3930 Visp, Switzerland'],
 ['Rheinfelden', 'Rheinfelden, Switzerland'],
 ['Glarus', '8750 Glarus, Switzerland'],
 ['Herisau', 'Herisau, Switzerland'],
 ['Laufen', '4242 Laufen, Switzerland'],
 ['Buchs', 'Buchs, Switzerland'],
 ['Losanna', 'Lausanne, Switzerland'],
 ['Altdorf', '6460 Altdorf, Switzerland'],
 ['Romanshorn', '8590 Romanshorn, Switzerland'],
 ['Sursee', 'Sursee, Switzerland'],
 ['Schmerikon', 'Schmerikon, Switzerland'],
 ['Sc

In [16]:
cantons = [[a[0], getCanton(a[1])] for a in addresses if a[1] != ""]

IndexError: list index out of range

In [ ]:
cantons

In [ ]:
df_cantons = pd.DataFrame(cantons)
df_cantons

In [ ]:
df_cantons.columns = [['Location','ID']]
df_cantons

In [ ]:
df_canton_name = pd.read_csv('data/canton_name.csv', sep=',') # Read the canton names

In [ ]:
df_canton_name

In [ ]:
df = pd.merge(df_canton_name, df_cantons, how='outer') # merge the two datasets to have all the cantons

In [ ]:
df

In [ ]:
df.to_csv('data/location_to_canton.csv')